# Detailed Run Analysis - Tabular Statistics

This notebook provides comprehensive tabular analysis of backtest runs, focusing on trade statistics, position duration, and drawdown metrics.

## Features

- Trade statistics: win rate, average win/loss, total trades
- Position holding duration analysis (in bars)
- Maximum drawdown calculations
- Comparative tables across multiple runs
- No heavy plotting - optimized for quick analysis

## Usage

1. Configure stock symbol
2. Scan and filter runs
3. Load selected runs
4. Review detailed statistics tables

Use this notebook when you need precise numbers and comparisons rather than visualizations.

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add analysis folder to path for imports
project_root = Path('/home/pjpr/projects/wee_hedgy_thing/quiescence')
analysis_path = project_root / 'analysis'

if str(analysis_path) not in sys.path:
    sys.path.insert(0, str(analysis_path))

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Import custom utility functions from analysis folder
# Force reload if already imported (useful during development)
import importlib
import utilities
importlib.reload(utilities)

from utilities import (
    scan_backtest_runs, 
    load_run_data, 
    create_runs_summary_dataframe, 
    convert_utc_to_ny,
    build_cumulative_pnl_from_positions
)


## 1. Configuration

Set the stock symbol and paths for analysis.

In [ ]:
# === CONFIGURATION ===
stock_symbol = "MSFT"

# Project paths
STORAGE_ROOT = Path("/data/quiescence/")
BACKTEST_ROOT = STORAGE_ROOT / "backtest"

print(f"Analyzing stock: {stock_symbol}")
print(f"Backtest runs root: {BACKTEST_ROOT}")

## 2. Scan and Filter Runs

Scan all available runs and create a summary DataFrame for filtering.

In [ ]:
# Scan all runs for the configured stock (fast - metadata only)
runs_metadata = scan_backtest_runs(BACKTEST_ROOT, stock_symbol)

# Create summary DataFrame for easy filtering and comparison
df_summary = create_runs_summary_dataframe(runs_metadata)

print(f"\nFound {len(df_summary)} runs for {stock_symbol}\n")
print(df_summary.to_string(index=False))

## 3. Select Runs to Load

Choose which runs to analyze.

In [ ]:
# Option 1: Load ALL runs
# runs_data = [load_run_data(run) for run in runs_metadata]

# Option 2: Load specific runs by Run number
#selected_run_numbers = [11, 19, 23, 34, 42, 43, 47]
#runs_data = [load_run_data(runs_metadata[run_num - 1]) for run_num in selected_run_numbers if 0 < run_num <= len(runs_metadata)]

# Option 3: Filter using DataFrame conditions
# Example: Load runs with specific parameter values
#filtered_df = df_summary[
#    (df_summary['Entry P Top'].astype(float) >= 0.95) & 
#    (df_summary['Frequency'] == '1-MINUTE')
#]

filtered_df = df_summary[(df_summary['Max Pos Bars'] == 9999)]

selected_indices = filtered_df['Run'].values - 1
runs_data = [load_run_data(runs_metadata[i]) for i in selected_indices]

print(f"\nLoaded {len(runs_data)} runs:\n")
print(filtered_df.to_string(index=False))

## 5. Trade Statistics Summary

Calculate comprehensive trading metrics for each run.

In [ ]:
# === TRADE STATISTICS CALCULATION ===
trade_stats = []

for i, run in enumerate(runs_data):
    df = run['data']
    positions = run['positions_report']
    
    # Extract bar frequency for display
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}-{parts[2]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    # Calculate statistics
    total_trades = len(positions)
    
    # Count winning/losing trades
    if total_trades > 0 and 'realized_pnl' in positions.columns:
        # Clean realized_pnl column
        if positions['realized_pnl'].dtype == 'object':
            realized_pnl_clean = positions['realized_pnl'].str.replace(' USD', '').astype(float)
        else:
            realized_pnl_clean = positions['realized_pnl']
        
        # Calculate win/loss metrics
        winning_trades = (realized_pnl_clean > 0).sum()
        losing_trades = (realized_pnl_clean < 0).sum()
        win_rate = winning_trades / total_trades if total_trades > 0 else 0
        
        avg_win = realized_pnl_clean[realized_pnl_clean > 0].mean() if winning_trades > 0 else 0
        avg_loss = realized_pnl_clean[realized_pnl_clean < 0].mean() if losing_trades > 0 else 0
        
        # Profit factor (sum of wins / abs(sum of losses))
        total_wins = realized_pnl_clean[realized_pnl_clean > 0].sum()
        total_losses = abs(realized_pnl_clean[realized_pnl_clean < 0].sum())
        profit_factor = total_wins / total_losses if total_losses > 0 else float('inf')
        
        # Calculate average holding time
        if 'duration' in positions.columns:
            avg_duration = positions['duration'].mean()
        else:
            avg_duration = None
        
        # Final P&L
        final_pnl = realized_pnl_clean.sum()
    else:
        winning_trades = losing_trades = 0
        win_rate = avg_win = avg_loss = avg_duration = final_pnl = profit_factor = None
    
    # Max drawdown calculation from cumulative P&L
    if final_pnl is not None:
        cumulative_pnl = build_cumulative_pnl_from_positions(positions)
        running_max = cumulative_pnl.expanding().max()
        drawdown = cumulative_pnl - running_max
        max_drawdown = drawdown.min()
        
        # Drawdown percentage (relative to peak)
        max_drawdown_pct = (max_drawdown / running_max.max() * 100) if running_max.max() > 0 else 0
    else:
        max_drawdown = max_drawdown_pct = None
    
    trade_stats.append({
        'Run': i + 1,
        'Frequency': frequency,
        'Max Pos Bars': run.get('max_position_bars', 'N/A'),
        'Total Trades': total_trades,
        'Winning': winning_trades,
        'Losing': losing_trades,
        'Win Rate (%)': f"{win_rate * 100:.1f}" if win_rate is not None else 'N/A',
        'Avg Win ($)': f"{avg_win:.2f}" if avg_win is not None and avg_win > 0 else 'N/A',
        'Avg Loss ($)': f"{avg_loss:.2f}" if avg_loss is not None and avg_loss < 0 else 'N/A',
        'Profit Factor': f"{profit_factor:.2f}" if profit_factor is not None and profit_factor != float('inf') else 'N/A',
        'Final P&L ($)': f"{final_pnl:.2f}" if final_pnl is not None else 'N/A',
        'Max DD ($)': f"{max_drawdown:.2f}" if max_drawdown is not None else 'N/A',
        'Max DD (%)': f"{max_drawdown_pct:.1f}" if max_drawdown_pct is not None else 'N/A',
        'Avg Duration': f"{avg_duration}" if avg_duration is not None else 'N/A',
    })

# Display as DataFrame
df_trade_stats = pd.DataFrame(trade_stats)
print("\n" + "="*130)
print("TRADE STATISTICS SUMMARY")
print("="*130 + "\n")
print(df_trade_stats.to_string(index=False))
print("\n" + "="*130)

## 6. Position Holding Duration Analysis

Analyze how long positions are held (measured in bars).

In [ ]:
# === POSITION DURATION ANALYSIS ===
duration_stats = []

for i, run in enumerate(runs_data):
    df = run['data']
    positions_report = run['positions_report']
    
    # Extract bar frequency
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}-{parts[2]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    # Ensure position column exists
    position_col = None
    for col in ['current_position_test', 'current_position', 'position']:
        if col in df.columns:
            position_col = col
            break
    
    if position_col:
        # Calculate position changes (when position != previous position)
        df['position_change'] = df[position_col].ne(df[position_col].shift())
        
        # Count bars in each position
        df['position_group'] = df['position_change'].cumsum()
        position_durations = df.groupby('position_group').size()
        
        # Exclude flat positions (position = 0) for duration analysis
        active_positions = df[df[position_col] != 0].groupby('position_group').size()
        
        if len(active_positions) > 0:
            avg_duration = active_positions.mean()
            median_duration = active_positions.median()
            max_duration = active_positions.max()
            min_duration = active_positions.min()
            std_duration = active_positions.std()
        else:
            avg_duration = median_duration = max_duration = min_duration = std_duration = 0
    else:
        avg_duration = median_duration = max_duration = min_duration = std_duration = None
    
    duration_stats.append({
        'Run': i + 1,
        'Frequency': frequency,
        'Max Pos Bars': run.get('max_position_bars', 'N/A'),
        'Avg Duration (bars)': f"{avg_duration:.1f}" if avg_duration is not None else 'N/A',
        'Median Duration (bars)': f"{median_duration:.0f}" if median_duration is not None else 'N/A',
        'Std Duration (bars)': f"{std_duration:.1f}" if std_duration is not None else 'N/A',
        'Max Duration (bars)': f"{max_duration}" if max_duration is not None else 'N/A',
        'Min Duration (bars)': f"{min_duration}" if min_duration is not None else 'N/A',
    })

# Display results
df_duration_stats = pd.DataFrame(duration_stats)
print("\n" + "="*110)
print("POSITION HOLDING DURATION ANALYSIS")
print("="*110 + "\n")
print(df_duration_stats.to_string(index=False))
print("\nNote: Duration measured in number of bars, not time units.")
print("="*110)

## 7. Drawdown Analysis Table

Detailed drawdown metrics for each run.

In [ ]:
# === DRAWDOWN ANALYSIS ===
drawdown_stats = []

for i, run in enumerate(runs_data):
    df = run['data']
    positions = run['positions_report']
    
    # Extract bar frequency
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}-{parts[2]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    # Build cumulative P&L
    pnl = build_cumulative_pnl_from_positions(positions)
    
    if len(pnl) > 0:
        # Calculate running maximum and drawdown
        running_max = pnl.expanding().max()
        drawdown = pnl - running_max
        
        # Find maximum drawdown
        max_dd = drawdown.min()
        max_dd_idx = drawdown.idxmin()
        
        # Find the peak before max drawdown
        peak_before_dd = running_max.loc[:max_dd_idx].idxmax()
        
        # Calculate drawdown duration (number of bars in drawdown)
        dd_duration = len(drawdown[peak_before_dd:max_dd_idx])
        
        # Calculate recovery (if recovered)
        after_dd = pnl[max_dd_idx:]
        peak_value = running_max.loc[max_dd_idx]
        recovered = (after_dd >= peak_value).any() if len(after_dd) > 0 else False
        
        if recovered:
            recovery_idx = after_dd[after_dd >= peak_value].index[0]
            recovery_duration = len(pnl[max_dd_idx:recovery_idx])
        else:
            recovery_duration = None
        
        # Drawdown percentage
        max_dd_pct = (max_dd / peak_value * 100) if peak_value > 0 else 0
        
        # Final P&L
        final_pnl = pnl.iloc[-1]
    else:
        max_dd = max_dd_pct = dd_duration = recovery_duration = final_pnl = None
    
    drawdown_stats.append({
        'Run': i + 1,
        'Frequency': frequency,
        'Max Pos Bars': run.get('max_position_bars', 'N/A'),
        'Final P&L ($)': f"{final_pnl:.2f}" if final_pnl is not None else 'N/A',
        'Max Drawdown ($)': f"{max_dd:.2f}" if max_dd is not None else 'N/A',
        'Max DD (%)': f"{max_dd_pct:.1f}" if max_dd_pct is not None else 'N/A',
        'DD Duration (bars)': f"{dd_duration}" if dd_duration is not None else 'N/A',
        'Recovery Duration (bars)': f"{recovery_duration}" if recovery_duration is not None else 'Not Recovered',
    })

# Display results
df_drawdown_stats = pd.DataFrame(drawdown_stats)
print("\n" + "="*110)
print("DRAWDOWN ANALYSIS")
print("="*110 + "\n")
print(df_drawdown_stats.to_string(index=False))
print("\n" + "="*110)

## 8. Combined Summary Report

All key metrics in one comprehensive table.

In [ ]:
# Merge all statistics into one comprehensive report
combined_stats = []

for i in range(len(runs_data)):
    # Get stats from each analysis
    trade_stat = trade_stats[i] if i < len(trade_stats) else {}
    duration_stat = duration_stats[i] if i < len(duration_stats) else {}
    drawdown_stat = drawdown_stats[i] if i < len(drawdown_stats) else {}
    
    combined_stats.append({
        'Run': i + 1,
        'Frequency': trade_stat.get('Frequency', 'N/A'),
        'Max Pos Bars': trade_stat.get('Max Pos Bars', 'N/A'),
        'Total Trades': trade_stat.get('Total Trades', 'N/A'),
        'Win Rate': trade_stat.get('Win Rate (%)', 'N/A'),
        'Profit Factor': trade_stat.get('Profit Factor', 'N/A'),
        'Final P&L': trade_stat.get('Final P&L ($)', 'N/A'),
        'Max DD': drawdown_stat.get('Max Drawdown ($)', 'N/A'),
        'Max DD %': drawdown_stat.get('Max DD (%)', 'N/A'),
        'Avg Duration': duration_stat.get('Avg Duration (bars)', 'N/A'),
    })

df_combined = pd.DataFrame(combined_stats)
print("\n" + "="*110)
print("COMBINED SUMMARY REPORT")
print("="*110 + "\n")
print(df_combined.to_string(index=False))
print("\n" + "="*110)

# Identify top performers
if len(df_combined) > 0:
    print("\n🏆 TOP PERFORMERS:\n")
    
    # Best final P&L
    df_combined['Final P&L (numeric)'] = df_combined['Final P&L'].str.replace('$', '').str.replace(',', '').astype(float, errors='ignore')
    best_pnl_idx = df_combined['Final P&L (numeric)'].idxmax()
    print(f"Highest P&L: Run {df_combined.loc[best_pnl_idx, 'Run']} with ${df_combined.loc[best_pnl_idx, 'Final P&L']}")
    
    # Best win rate
    df_combined['Win Rate (numeric)'] = df_combined['Win Rate'].str.replace('%', '').astype(float, errors='ignore')
    best_wr_idx = df_combined['Win Rate (numeric)'].idxmax()
    print(f"Highest Win Rate: Run {df_combined.loc[best_wr_idx, 'Run']} with {df_combined.loc[best_wr_idx, 'Win Rate']}")
    
    # Best profit factor
    df_combined['Profit Factor (numeric)'] = df_combined['Profit Factor'].astype(float, errors='ignore')
    best_pf_idx = df_combined['Profit Factor (numeric)'].idxmax()
    print(f"Best Profit Factor: Run {df_combined.loc[best_pf_idx, 'Run']} with {df_combined.loc[best_pf_idx, 'Profit Factor']}")